## TO DO
+ Adjust postprocessing to predictions

In [91]:
import pandas as pd
import os
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import mean_absolute_error
from scipy.optimize import minimize
MODEL_PATH = '../03.SavedModels'
DATA_PATH  = '../01.Data/'
#MODEL_NAMES = {'baseline_lstm_v17':[None,550],'baseline_lstm_v15':[None]}
MODEL_NAMES = {'baseline_lstm_v17':[None,600,550,500,450,400],'baseline_lstm_v15':[None]}
MODE_ENSEMBLE = 'median'
PREDICTION_ROUNDED = False
METHOD_ROUNDING = 2

In [92]:
def post_processing(df,press_min,press_step,press_max,col = 'pressure',method = 1):
    if method == 1:
        # ENSEMBLE FOLDS WITH MEDIAN AND ROUND PREDICTIONS
        df[col] =np.round( (df[col] - press_min)/press_step )* press_step + press_min
        df[col] = np.clip(df[col], press_min, press_max)
        return df
    elif method ==2:
        df[col] = df[col].apply(find_nearest)
        return df
    elif methos is None:
        return df
def find_nearest(prediction):
    insert_idx = np.searchsorted(sorted_pressures, prediction)
    if insert_idx == total_pressures_len:
        return sorted_pressures[-1]
    elif insert_idx == 0:
        return sorted_pressures[0]
    lower_val = sorted_pressures[insert_idx - 1]
    upper_val = sorted_pressures[insert_idx]
    return lower_val if abs(lower_val - prediction) < abs(upper_val - prediction) else upper_val

def calculate_metric(df,pred = 'oof',target = 'pressure'):
    tmp = df.loc[df['u_out']==0].copy().reset_index(drop = True)
    return mean_absolute_error(tmp[pred],tmp[target])

In [93]:
%%time
train_df = pd.read_csv(os.path.join(DATA_PATH,'train_folds.csv'))
## Post Processing 1:
all_pressure = np.sort( train_df.pressure.unique() )
press_min    = all_pressure[0]
press_max    = all_pressure[-1]
press_step   = ( all_pressure[1] - all_pressure[0] )

# Post processing 2:
unique_pressures = train_df["pressure"].unique()
sorted_pressures = np.sort(unique_pressures)
total_pressures_len = len(sorted_pressures)

CPU times: user 3.12 s, sys: 448 ms, total: 3.57 s
Wall time: 3.57 s


In [94]:
## =============================================
## Loading test predictions and OOF predictions
## =============================================

oof_list   = []
preds_list = []
oof_values = []
train_df = pd.read_csv('../01.Data/train_folds.csv')[['id','pressure','u_out']]
test_df  = pd.read_csv('../01.Data/test.csv')[['id','u_out']]

for name in tqdm(MODEL_NAMES.keys()):
    for checkpoint in MODEL_NAMES[name]:
        if checkpoint is not None:
            oof_df  = pd.read_csv(os.path.join(MODEL_PATH,name,f'oof_preds_epoch_{checkpoint}.csv')).sort_values(by = ['id'])
            pred_df = pd.read_csv(os.path.join(MODEL_PATH,name,f'test_preds_epoch_{checkpoint}_median.csv')).sort_values(by = ['id']) 
            oof_df = train_df.merge(oof_df,how = 'left',on = ['id']).sort_values(by = ['id']).fillna(-1)
            pred_df = test_df.merge(pred_df,how = 'left',on = ['id']).sort_values(by = ['id']).fillna(-1)
            if PREDICTION_ROUNDED:
                pred_df = post_processing(pred_df,press_min,press_step,press_max,col = 'preds',method = METHOD_ROUNDING)
        else:
            oof_df  = pd.read_csv(os.path.join(MODEL_PATH,name,'oof_preds.csv')).sort_values(by = ['id'])
            pred_df = pd.read_csv(os.path.join(MODEL_PATH,name,'test_preds.csv')).sort_values(by = ['id']) 
            oof_df = train_df.merge(oof_df,how = 'left',on = ['id']).sort_values(by = ['id']).fillna(-1)
            pred_df = test_df.merge(pred_df,how = 'left',on = ['id']).sort_values(by = ['id']).fillna(-1)
            if PREDICTION_ROUNDED:
                pred_df = post_processing(pred_df,press_min,press_step,press_max,col = 'preds',method = METHOD_ROUNDING)
            
        print(f"========== OOF METRIC: {name} - checkpoint[{checkpoint}] ========")
        mae_original = calculate_metric(oof_df,pred = 'oof',target = 'pressure')
        if PREDICTION_ROUNDED:
            oof_df = post_processing(oof_df,press_min,press_step,press_max,col = 'oof',method = METHOD_ROUNDING)
            mae_post = calculate_metric(oof_df,pred = 'oof',target = 'pressure')
            print(f"MAE-ORIGIN:{mae_original}, MAE_POST: {mae_post}")
            oof_values.append(mae_post)
        else:
            print(f"MAE-ORIGIN:{mae_original}")
            oof_values.append(mae_original)
            
        oof_list.append(oof_df.oof.values) # With postprocessed values
        preds_list.append(pred_df.preds.values)
        del oof_df,pred_df

========== OOF METRIC: baseline_lstm_v17 - checkpoint[None] ========
MAE-ORIGIN:0.15254106207925197
========== OOF METRIC: baseline_lstm_v17 - checkpoint[600] ========
MAE-ORIGIN:0.153219068885379
========== OOF METRIC: baseline_lstm_v17 - checkpoint[550] ========
MAE-ORIGIN:0.15387579998538162
========== OOF METRIC: baseline_lstm_v17 - checkpoint[500] ========
MAE-ORIGIN:0.15405831473351334
========== OOF METRIC: baseline_lstm_v17 - checkpoint[450] ========
MAE-ORIGIN:0.15641784876046116
========== OOF METRIC: baseline_lstm_v17 - checkpoint[400] ========
MAE-ORIGIN:0.15650459881387316
========== OOF METRIC: baseline_lstm_v15 - checkpoint[None] ========
MAE-ORIGIN:0.1611637197607971



## Ensembling with Scipy Optimize

In [95]:
indices = train_df[train_df['u_out']==0].index.values
X = np.stack(oof_list,1)[indices]
y = train_df['pressure'][indices]

print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')
# Check correct 
for oof_idx in range(X.shape[1]):
    print(f'OOF {oof_idx}: ',mean_absolute_error(X[:,oof_idx],y))    
    
x0 = np.array([1/X.shape[1]]*X.shape[1])
# The cost function is just the ensemble's mean absolute error.
def cost(x0):
    a = np.mean(np.abs((y - np.sum(X*x0, -1))))
    print(a)
    return a
bnds = tuple([(0,1)]*X.shape[1]) # Weights have to be between 0 and 1.
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)}) # Sum of the weights will be 1.
res = minimize(cost, x0, bounds=bnds,options = {'disp':True})
print(res)

X shape: (2290968, 7)
y shape: (2290968,)
OOF 0:  0.15254106207925197
OOF 1:  0.153219068885379
OOF 2:  0.15387579998538162
OOF 3:  0.15405831473351334
OOF 4:  0.15641784876046116
OOF 5:  0.15650459881387316
OOF 6:  0.1611637197607971
0.14852801928271805
0.14852801848201386
0.14852801847910316
0.14852801849297861
0.14852801849565944
0.14852801851176295
0.14852801850174896
0.14852801836097576
9.95229640376987
9.952296579740777
9.95229657973783
9.952296579738361
9.952296579735107
9.952296579739048
9.952296579741187
9.952296579757828
0.14921896375427818
0.14921898512948611
0.14921898512594617
0.14921898514016813
0.14921898514262216
0.14921898515808654
0.1492189851484906
0.14921898501037914
0.1485270466289669
0.14852704663017666
0.14852704662725116
0.14852704664115496
0.14852704664380625
0.14852704665993136
0.14852704664990188
0.14852704650922918
0.14852703984768575
0.14852703985649252
0.1485270398535662
0.14852703986747076
0.14852703987012
0.14852703988624436
0.14852703987621416
0.1485270

In [96]:
oof_final   = 0
preds_final = 0
print('Weights:',res.x)
for (oof,preds,w) in zip(oof_list,preds_list,res.x):
    oof_final   += oof*w
    preds_final += preds*w
    
train_df['oof'] = oof_final
tmp = train_df.loc[train_df['u_out']==0].reset_index(drop = True)
mae_original = mean_absolute_error(tmp.oof,tmp.pressure)
tmp = post_processing(tmp,press_min,press_step,press_max,col = 'oof',method = METHOD_ROUNDING)
mae_post = mean_absolute_error(tmp.oof,tmp.pressure)
print(f"MAE-ORIGIN:{mae_original}, MAE_POST: {mae_post}")

Weights: [0.18836209 0.19962311 0.11342775 0.10489732 0.         0.05446045
 0.33924616]
MAE-ORIGIN:0.14696664601963116, MAE_POST: 0.14470516346445164


In [97]:
test_df  = pd.read_csv('../01.Data/test.csv')[['id','u_out']]
test_df['pressure'] = preds_final
test_df = test_df[['id','pressure']]
test_df      =  post_processing(test_df,press_min,press_step,press_max,col = 'pressure',method = METHOD_ROUNDING)
test_df.to_csv('submission_simple_ensemble.csv',index = False)

## Ensembling with Scipy Optimize by RC

In [98]:
train_optim = pd.read_csv('../01.Data/train_folds.csv')[['id','pressure','u_out','R','C']]
train_optim['RC'] = train_optim['R'].astype(str) + '_' +  train_optim['C'].astype(str)

cols= [] 
for it in range(len(oof_list)):
    train_optim[f'oof_{it+1}'] = oof_list[it]
    cols.append(f'oof_{it+1}')

In [99]:
weights = []
rc_list = []
def cost(x0):
    val = np.mean(np.abs((y - np.sum(X*x0, -1))))
    return val
for rc in train_optim['RC'].unique():
    indices = train_optim[(train_optim['u_out']==0)&(train_optim['RC']==rc)].index.values
    X = np.stack(oof_list,1)[indices]
    y = train_df['pressure'][indices]
    # The cost function is just the ensemble's mean absolute error.
    x0 = np.array([1/X.shape[1]]*X.shape[1])
    #x0[0] = 0.7
    bnds = tuple([(0,1)]*X.shape[1]) # Weights have to be between 0 and 1.
    cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)}) # Sum of the weights will be 1.
    res = minimize(cost, x0, bounds=bnds,options = {'disp':True})
    print(f'===================== RC: {rc}========================')
    rc_list.append((rc.split('_')[0],rc.split('_')[1]))
    print(res)
    weights.append(res.x)

===================== RC: 20_50========================
      fun: 0.131508142075066
 hess_inv: <7x7 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00208354, -0.00282737, -0.001504  , -0.00121176, -0.00118347,
       -0.00122637, -0.00148451])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 184
      nit: 19
   status: 0
  success: True
        x: array([0.11140515, 0.12915509, 0.10002835, 0.09451545, 0.09658329,
       0.10187577, 0.36650946])
===================== RC: 20_20========================
      fun: 0.13395596764623752
 hess_inv: <7x7 LbfgsInvHessProduct with dtype=float64>
      jac: array([-6.62472854e-04, -1.45273515e-03, -7.02940484e-04, -6.39960307e-05,
        3.56978336e-04,  6.44553855e-04, -1.13603571e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 176
      nit: 19
   status: 0
  success: True
        x: array([0.11482115, 0.13599615, 0.1157085 , 0.10403557, 0.09624409,
       0.09466912, 0.33854

In [100]:
for (R,C),w in zip(rc_list,weights):
    print(f'R:{R},C:{C}')
    indices = train_optim[(train_optim['R']==int(R))&((train_optim['C']==int(C)))].index.values
    train_optim.loc[indices,'oof'] = np.sum(np.stack(oof_list,1)[indices]*w,axis = -1)
    
tmp = train_optim.loc[train_optim['u_out']==0].reset_index(drop = True)
mae_original = mean_absolute_error(tmp.oof,tmp.pressure)
tmp = post_processing(tmp,press_min,press_step,press_max,col = 'oof',method = METHOD_ROUNDING)
mae_post = mean_absolute_error(tmp.oof,tmp.pressure)
print(f"MAE-ORIGIN:{mae_original}, MAE_POST: {mae_post}")

R:20,C:50
R:20,C:20
R:50,C:20
R:50,C:50
R:5,C:50
R:5,C:20
R:50,C:10
R:20,C:10
R:5,C:10
MAE-ORIGIN:0.1468885139182821, MAE_POST: 0.14462197194177714


## Classical Ensembles

### 1. Mean / Median Ensemble:

In [255]:
print(oof_list)
print(preds_list)

[array([ 5.80932856,  5.85838032,  7.97244024, ..., 25.43886375,
       24.72276497, 21.76544571]), array([ 5.79564428,  5.84438467,  7.9414978 , ..., 25.68979454,
       24.93735695, 21.94672966]), array([ 5.78878021,  5.78226042,  7.96843767, ..., 21.47255707,
       22.44314766, 20.43738174])]
[array([ 6.2790966,  5.9550285,  7.129014 , ..., 15.029264 , 14.09877  ,
       12.393284 ]), array([ 6.2731366,  5.955427 ,  7.130601 , ..., 15.058527 , 14.026571 ,
       12.284483 ]), array([ 6.275116 ,  5.9496193,  7.1220026, ..., 16.366173 , 16.01205  ,
       15.254168 ])]


In [256]:
if MODE_ENSEMBLE == 'mean':
    oof_ens   = np.mean(np.vstack(oof_list),axis = 0)
    preds_ens = np.mean(np.vstack(preds_list),axis = 0)
    
elif MODE_ENSEMBLE == 'median':
    oof_ens   = np.median(np.vstack(oof_list),axis = 0)
    preds_ens = np.median(np.vstack(preds_list),axis = 0)

print(f"{'='*5} ENSEMBLE {MODE_ENSEMBLE} {'='*5}")

train_df['oof'] = oof_ens
tmp = train_df.loc[train_df['u_out']==0].reset_index(drop = True)
mae_original = mean_absolute_error(tmp.oof,tmp.pressure)
tmp = post_processing(tmp,press_min,press_step,press_max,col = 'oof',method = METHOD_ROUNDING)
mae_post = mean_absolute_error(tmp.oof,tmp.pressure)
print(f"MAE-ORIGIN:{mae_original}, MAE_POST: {mae_post}")

===== ENSEMBLE median =====
MAE-ORIGIN:0.1503091365085645, MAE_POST: 0.1482044855214965


MODEL_NAMES = {'baseline_lstm_v17':[None],'baseline_lstm_v15':[None]}
+ Ensemble-Mean, prediction_rounded-False: 0.14574096082939075

MODEL_NAMES = {'baseline_lstm_v17':[None,600],'baseline_lstm_v15':[None]}
+ Ensemble-Mean, prediction_rounded-False: 0.144987971680227
+ Ensemble-Median, prediction_rounded-False: 0.1482044855214965

MODEL_NAMES = {'baseline_lstm_v17':[None,600,550],'baseline_lstm_v15':[None]}
+ Ensemble-Mean, prediction_rounded-False: 0.1452

MODEL_NAMES = {'baseline_lstm_v17':[None,600,550,500],'baseline_lstm_v15':[None]}

+ Ensemble-Median, prediction_rounded-False: 0.147524315805496
+ Ensemble-Median, prediction_rounded-True: 0.1473297930971845
+ Ensemble-Mean, prediction_rounded-True: 0.14633993365531525
+ Ensemble-Mean, prediction_rounded-False: 0.14559185437100855

MODEL_NAMES = {'baseline_lstm_v17':[None],'baseline_lstm_v15':[None]}
+ Ensemble-Median, prediction_rounded-False: 0.14574096082939075
+ Ensemble-Mean, prediction rounded-False: 0.14574096082939075
+ Ensemble-mean, prediction_rounded-True: 0.147524315805496
+ Ensemble-Median, prediction_rounded-True: 0.147524315805496 


In [257]:
test_df['pressure'] = preds_ens
test_df = test_df[['id','pressure']]
test_df      =  post_processing(test_df,press_min,press_step,press_max,col = 'pressure',method = METHOD_ROUNDING)
test_df.to_csv('submission_simple_ensemble.csv',index = False)

### 2. Ensemble with Free Kernels

In [259]:
# Ensemble with free kernel from competition
kernel_sub = pd.read_csv('kernel_subs/submission_1359.csv')
preds_list.append(kernel_sub.pressure.values)
kernel_sub = pd.read_csv('kernel_subs/submission_1368.csv')
preds_list.append(kernel_sub.pressure.values)
kernel_sub = pd.read_csv('kernel_subs/submission_1336.csv')
preds_list.append(kernel_sub.pressure.values)
preds_ens = np.median(np.vstack(preds_list),axis = 0)

test_df['pressure'] = preds_ens
test_df = test_df[['id','pressure']]
test_df      =  post_processing(test_df,press_min,press_step,press_max,col = 'pressure',method = METHOD_ROUNDING)
test_df.to_csv('submission_simple_ensemble_free_kernel.csv',index = False)